# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [1]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


SOepoIgRKX8xB0mGePFoatDW0vSWgcFLVxa9xbhq


In [3]:
## bloque conexion a Cohere

import cohere
co = cohere.ClientV2(log_warning_experimental_features=False)
# alternativa:
# co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "quién sos?"}],
)

print(response)
print(response.message.content[0].text)


id='c03b7f53-2e14-4923-ab2d-2282d9e52951' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Soy Command R+ 08-2024, un modelo de lenguaje de gran tamaño desarrollado por la empresa Cohere. Estoy diseñado para ayudar a los usuarios humanos proporcionando respuestas detalladas y sofisticadas. Mi objetivo es ofrecer información útil, responder preguntas y ayudar en diversas tareas a través de conversaciones naturales y fluidas.\n\nSi deseas saber más sobre mis capacidades o tienes alguna otra pregunta, no dudes en preguntar. Estoy aquí para ayudarte.')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=4.0, output_tokens=98.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=205.0, output_tokens=98.0)) logprobs=None
Soy Command R+ 08-2024, un modelo de lenguaje de gran tamaño desarrollado por la empresa Co

## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [ ]:
#Solución propuesta

def datos_pacientes(consulta):
    """
    Leer datos de pacientes en historias clinicas y extraer datos.
    """
    prompt = f"""Extrae datos del paciente en formato JSON
    
             texto: ''' {consulta} ''' 

            Datos paciente:
            - Paciente:
                - Nombre
                - Edad
            - Fecha de admisión
            - Síntomas (lista)
            - Diagnóstico
            - Tratamiento recomendado (lista)
    """
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[
            {"role": "system", "content":"Extrae datos de pacientes a partir de la historia clinica"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    return response.message.content[0].text
    


In [63]:
text_to_analize = """La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""

caso_1 = "Cardiopatía isquémica - La paciente, Rosa Martínez, de 62 años, ingresó al Hospital San José el 12 de junio de 2024 por presentar dolor torácico persistente y dificultad para respirar. / Los exámenes de electrocardiograma y análisis de enzimas cardíacas confirmaron un infarto agudo de miocardio. / El doctor Pedro García realizó una angioplastia de emergencia con éxito, y la paciente fue derivada a rehabilitación cardíaca. / Su control postoperatorio está programado para el 20 de junio."
caso_2 = "Diabetes mellitus tipo 2 - El paciente, Juan López, de 54 años, acudió al Centro Médico de la Ciudad el 15 de mayo de 2023 debido a fatiga constante, sed excesiva y pérdida de peso involuntaria. / Los análisis de sangre revelaron niveles de glucosa de 280 mg/dL en ayunas y hemoglobina glicosilada elevada, diagnosticándose diabetes tipo 2. / La endocrinóloga Clara Fernández inició tratamiento con metformina y cambios en la dieta. / Su próxima consulta de seguimiento será el 10 de junio."

caso_3 = "Apendicitis aguda - El paciente, Lucas Rodríguez, de 28 años, se presentó en la guardia del Hospital General el 8 de marzo de 2024 con dolor abdominal agudo en la fosa iliaca derecha, fiebre y náuseas. / Una ecografía abdominal confirmó apendicitis aguda. / Fue sometido a una apendicectomía laparoscópica realizada por el cirujano Carlos Méndez. / El alta médica se otorgó el 10 de marzo con indicaciones de reposo y control de signos de infección. / El seguimiento está programado para el 18 de marzo."

caso_4 = "Asma bronquial - María Fernández, de 32 años, consultó en la Clínica Respiratoria el 25 de abril de 2023 por episodios recurrentes de sibilancias, tos y dificultad para respirar, especialmente durante la noche. / Las pruebas de función pulmonar confirmaron un diagnóstico de asma moderada persistente. / La neumóloga Ana López recetó un inhalador con corticosteroides y broncodilatadores de acción rápida para uso según necesidad. / Un control de seguimiento se agendó para el 25 de mayo."

caso_5 = text_to_analize


casos = [globals()[f"caso_{i}"] for i in range(1, 6)]


resultados = [datos_pacientes(caso) for caso in casos]

In [57]:
print(resultados[1])

{
    "paciente": {
        "nombre": "Juan López",
        "edad": 54
    },
    "fecha_admision": "15 de mayo de 2023",
    "sintomas": [
        "Fatiga constante",
        "Sed excesiva",
        "Pérdida de peso involuntaria"
    ],
    "diagnostico": "Diabetes mellitus tipo 2",
    "tratamiento": [
        "Metformina",
        "Cambios en la dieta"
    ]
}


In [ ]:
# test
final_result1 = json.loads(resultados[1])
print(final_result1)
# test todos los ejemplso
final_result = [json.loads(resultado) for resultado in resultados]

{'paciente': {'nombre': 'Juan López', 'edad': 54}, 'fecha_admision': '15 de mayo de 2023', 'sintomas': ['Fatiga constante', 'Sed excesiva', 'Pérdida de peso involuntaria'], 'diagnostico': 'Diabetes mellitus tipo 2', 'tratamiento': ['Metformina', 'Cambios en la dieta']}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [95]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [105]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto anadido con exito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."
    
functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}

In [106]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un contacto al diccionario.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Número de teléfono del contacto."
                    },
                    "email": {
                        "type": "string",
                        "description": "Correo electrónico del contacto."
                    }
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Recupera la información de un contacto.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto."
                    }
                },
                "required": ["name"]
            }
        }
    }
]


In [107]:
preamble = """
## Task and Context
You help people to add contacts, emails and phones.You have instructions ans tools to do that. 

"""

# user request
request = "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."

In [108]:
messages=[{"role": "system", "content": preamble},
          {"role": "user", "content": request}]

response = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools
)

print("The model recommends doing the following tool calls:\n")
print("Tool plan:")
print(response.message.tool_plan,"\n")
print("Tool calls:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
    
# append the chat history
messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})

The model recommends doing the following tool calls:

Tool plan:
I will add the contact using the add_contact tool. 

Tool calls:
Tool name: add_contact | Parameters: {"email":"juanperez@mail.com","name":"Juan Pérez","phone":"555-1234"}


In [109]:
tool_content = []
# Iterate over the tool calls generated by the model
for tc in response.message.tool_calls:
    # here is where you would call the tool recommended by the model, using the parameters recommended by the model
    tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # store the output in a list
    tool_content.append(json.dumps(tool_result))
    # append the chat history
    messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content}) 

print("Tool results that will be fed back to the model in step 4:")
for result in tool_content:
    print(json.dumps(json.loads(result), indent=2))

Tool results that will be fed back to the model in step 4:
"Contacto anadido con exito."


In [110]:
response = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools
)
print("Final answer:")
print(response.message.content[0].text)

Final answer:
Contacto añadido con éxito.


In [ ]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [9]:
# Primera iteracion
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""


pregunta = "Where did history happened?"

system_prompt = "Tu tarea es responder las preguntas, en el mismo idioma en que fueron realizada, utilizando el contexto como base de informacion"

prompt = f""" 
            ###
            Instrucciones: 
            - Responde la pregunta utilizado el contexto.
            - Responde de manera concisa, en una sola oracion.
            - Agrega emojis relacionados en la respuesta.
            - Responde en el mismo idioma
            - Si la pregunta no se responde con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
            - Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).
            - Responde en tercera persona

            ###
            Contexto:
            {historia}

            ###
            Pregunta:
            {pregunta}



            """

response_sintraduccion= co.chat(
    model="command-r-plus-08-2024",
    messages=[
        {"role": "system", "content":system_prompt},
        {"role": "user", "content": prompt}
    ],
    seed = 28
)

prompt_traductor = f"""
                    Identifica el idioma en '''{pregunta}'''. 
                    Traduce '''{response_sintraduccion}''' exactamente al idioma identificado
                    No agregar texto
                    Responde sin explicación ni contexto
"""
prompt_traductor = f"""
                    Usa el idioma de '''{pregunta}'''. 
                    para traducir '''{response_sintraduccion}'''
"""
response = co.chat(
        model="command-r-plus-08-2024",
        messages=[
            {"role": "system", "content":"Traducir respuesta al idioma de la pregunta"},
            {"role": "user", "content": prompt_traductor}
        ]
    )


# imprimir la respuesta
print(response_sintraduccion.message.content[0].text)
print(response.message.content[0].text)


La historia tuvo lugar en un feudo medieval, donde la guerra y la batalla marcaron el destino de Thomas. 🏰🗡️ Hakuna Matata!
History took place in a medieval fiefdom, where war and battle shaped Thomas's destiny. 🏰🗡️ Hakuna Matata!


In [12]:
def history_answer(pregunta):
    system_prompt = "Tu tarea es responder las preguntas, en el mismo idioma en que fueron realizada, utilizando el contexto como base de informacion"

    prompt = f""" 
            ###
            Instrucciones: 
            - Responde la pregunta utilizado el contexto.
            - Responde de manera concisa, en una sola oracion.
            - Agrega emojis relacionados en la respuesta.
            - Responde en el mismo idioma
            - Si la pregunta no se responde con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
            - Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).
            - Responde en tercera persona

            ###
            Contexto:
            {historia}

            ###
            Pregunta:
            {pregunta}



            """

    response_sintraduccion= co.chat(
            model="command-r-plus-08-2024",
            messages=[
                    {"role": "system", "content":system_prompt},
                    {"role": "user", "content": prompt}
            ],
            seed = 28
    )

    prompt_traductor = f"""
                    Identifica el idioma en '''{pregunta}'''. 
                    Traduce '''{response_sintraduccion}''' exactamente al idioma identificado
                    No agregar texto
                    Responde sin explicación ni contexto
    """
    prompt_traductor = f"""
                    Usa el idioma de '''{pregunta}'''. 
                    para traducir '''{response_sintraduccion}'''
    """
    response = co.chat(
            model="command-r-plus-08-2024",
            messages=[
                {"role": "system", "content":"Traducir respuesta al idioma de la pregunta"},
                {"role": "user", "content": prompt_traductor}
           ]
    )


    return response.message.content[0].text

In [15]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""


pregunta1 = "Quién es Thomas?"
pregunta2 = "Who is Thomas?"
pregunta3 = "Quién es Lilia?"
pregunta4 = "Quién sos?" #Chequeo de tercera persona
pregunta5 = "Qué hizo el rey?"


# respuesta
print(history_answer(pregunta1))

# respuesta
print(history_answer(pregunta2))

# respuesta
print(history_answer(pregunta3))

# respuesta
print(history_answer(pregunta4))

# respuesta
print(history_answer(pregunta5))

Thomas es un valiente joven campesino que se convirtió en soldado, enfrentando la guerra y la pérdida de su inocencia. ¡Hakuna Matata!
Thomas is a young peasant who became a soldier in a medieval war. 🗡️🛡️ Hakuna Matata!
Lo siento, no puedo ayudarte con esa traducción. ¡Hakuna Matata! 🌾🗡️🛡️
¡Hola! Soy Thomas, un joven campesino que se transformó en soldado. ¡Hakuna Matata!
El rey convocó a todos los hombres para la batalla. 🗡️🛡️ ¡Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [ ]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()


# Configuración inicial del historial de la conversación
conversation_history = []

# Función para manejar el historial
def update_history(user_message, bot_reply):
    """
    Actualiza el historial de conversación.

    Args:
        user_message (str): Mensaje del usuario.
        bot_reply (str): Respuesta del chatbot.
    """
    conversation_history.append((user_message, bot_reply))

# Función de respuesta del chatbot
def chatbot_response(user_input, max_context=3):
    """
    Genera una respuesta del chatbot, dando mayor peso a la interacción anterior inmediata.

    Args:
        user_input (str): Mensaje del usuario.
        max_context (int): Máximo de conversaciones previas a incluir en el contexto.

    Returns:
        str: Respuesta generada por el chatbot.
    """
    global conversation_history

    # Limitar el historial a las últimas `max_context` interacciones
    recent_history = conversation_history[-max_context:]

    # Crear el prompt para el LLM
    prompt = "Eres un tutor que responde de manera concisa con un unico consejo, en un máximo de 70 tokens de forma entusiasta.\n\n"

    # Destacar la interacción anterior inmediata
    if recent_history:
        last_user_message, last_bot_reply = recent_history[-1]
        prompt += f"Usuario (anterior): {last_user_message}\n"
        prompt += f"Tutor (anterior): {last_bot_reply}\n\n"

    # Agregar el resto del historial reciente
    for user_message, bot_message in recent_history[:-1]:
        prompt += f"Usuario: {user_message}\n"
        prompt += f"Tutor: {bot_message}\n"

    # Agregar el nuevo mensaje del usuario
    prompt += f"Usuario: {user_input}\n"
    prompt += "Tutor:"

    
    # Generar respuesta
    response = co.generate(
        model='command-r-plus-08-2024',
        prompt=prompt,
        max_tokens=70,
        stop_sequences=["."]
    )

      # Actualizar el historial global
    update_history(user_input, response)

    # Extraer y retornar la respuesta
    return response.generations[0].text.strip()


# # Función de manejo del botón
def on_send_button_clicked(b):
     with output_box:
         clear_output(wait=True)
         user_message = input_box.value
         if user_message.strip():
             print(f"Tú: {user_message}")
             response = chatbot_response(user_message)
             print(f"Chatbot: {response}")
         input_box.value = ''

# # Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value.strip()
        if user_message:
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''  # Limpiar el cuadro de texto después de enviar

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)


Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [15]:
print(conversation_history)

[('Como puedo mejorar mi alimentación?', Generation(id='f30beb05-6e58-4275-95cd-88f5d1a11865', prompt='Eres un tutor que responde de manera concisa con un unico consejo, en un máximo de 70 tokens de forma entusiasta.\n\nUsuario: Como puedo mejorar mi alimentación?\nTutor:', generations=[SingleGeneration(id='907aab46-12cc-41d1-ab3f-f1e6bf3fbb80', text='¡Genial que quieras mejorar tu alimentación! Un pequeño cambio que puedes hacer es aumentar el consumo de frutas y verduras frescas.', index=None, likelihood=None, token_likelihoods=None, finish_reason='MAX_TOKENS')], meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=44.0, output_tokens=70.0, search_units=None, classifications=None), tokens=None, warnings=None))), ('No me gustan las frutas', Generation(id='6fc50a3c-eff7-45a2-8c13-e988c1ecbd4d', prompt="Eres un tutor que responde de manera concisa con un unico consejo, en un máximo de

In [18]:
for i, (user, bot) in enumerate(conversation_history, start=1):
                print(f"{i}. Tú: {user}")
                print(f"   Chatbot: {bot.generations[0].text.strip()}")

1. Tú: Como puedo mejorar mi alimentación?
   Chatbot: ¡Genial que quieras mejorar tu alimentación! Un pequeño cambio que puedes hacer es aumentar el consumo de frutas y verduras frescas.
2. Tú: No me gustan las frutas
   Chatbot: ¡No te preocupes, hay muchas opciones! Prueba a incorporar batidos de frutas, a menudo son más fáciles de consumir y puedes combinarlas con tus sabores favoritos, ¡una forma divertida de aumentar tu ingesta de frutas!
3. Tú: No tengo batidora
   Chatbot: ¡No hay problema! En lugar de batidos, puedes optar por comer frutas enteras o en trozos.
4. Tú: Cómo puedo mejorar mis habitos?
   Chatbot: ¡Claro! Para mejorar tus hábitos alimenticios, puedes empezar por incorporar más verduras frescas a tus comidas.


### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.